# Fullcustom analog design demo using Magic

The environment path must be retrieved in order to provide _Magic_ with the technology files:

In [ ]:
import os
CONDA_PREFIX = os.environ['CONDA_PREFIX']

## Source files

### Using the _Magic_ VLSI layout tool

[_Magic_](http://opencircuitdesign.com/magic/) cells (`.mag`) are used to store and manage the layouts. They can be edited using _Magic_ commands or using _Magic_'s GUI.

An existing _inverter_ design as well as its dependencies is imported and converted to _Graphic Design System_ (`.gds`) file for viewing and manufacturing and a _SPICE_ (`.spice`) file for simulation:

In [ ]:
%%script magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc

load cad/mag/inverter.mag

extract
ext2spice lvs
ext2spice cthresh 0
ext2spice rthresh 0
ext2spice -o cad/spice/inverter.spice

gds labels no
gds write cad/gds/inverter.gds

### Display the inverter layout

The `.gds` file can be converted to vector graphics for displaying:

In [ ]:
import gdstk
import IPython.display

library = gdstk.read_gds('cad/gds/inverter.gds')
top_cells = library.top_level()
top_cells[0].write_svg('cad/svg/inverter.svg', scaling=100)
IPython.display.SVG('cad/svg/inverter.svg')

## Simulation

### Using _SPICE_ simulation engine

[_PySPICE_](https://pyspice.fabrice-salvaire.fr/releases/v1.4/overview.html) is a _Python_ implementation of several _SPICE_ simulation engines. The _SPICE_ engines are used to simulate analog circuits. A top-level test bench circuit must be built:

In [ ]:
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Unit import *

# Declare the top-level circuit
circuit = Circuit('Inverter')
# Link to technology libraries
circuit.lib(f'{CONDA_PREFIX}/share/pdk/sky130A/libs.tech/ngspice/sky130.lib.spice', 'tt')

# Import and instantiate the CUT (Circuit Under Test)
circuit.include('cad/spice/inverter.spice')
circuit.X('Inverter', 'inverter', 'VDD','VSS','I','O');

# Add voltage sources
circuit.V('VSS', 'VSS', 0, 0)
circuit.V('VDD', 'VDD', 'VSS', 1.8)
# Add pulse source
circuit.PulseVoltageSource(
    'I', 'I', 'VSS',
    initial_value=0@u_V, pulsed_value=1.8@u_V,
    rise_time=10@u_ps, fall_time=10@u_ps,
    pulse_width=40@u_ps, period=160@u_ps, delay_time=10@u_ps
)

# Create the simulation and simulate
simulator = circuit.simulator()
analysis = simulator.transient(step_time=1@u_ps, end_time=160@u_ps)

### Plot the waveform

The output waveform can be plotted:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(8,5))
ax.grid(visible=True, which='major', axis='both');
ax.grid(visible=True, which='minor', axis='both', linestyle="--");
ax.set_yticks(np.linspace(-1.0,2.5,8,endpoint=True));
ax.set_yticks(np.linspace(-1.0,2.5,36,endpoint=True),minor=True);
ax.set_xticks(np.linspace(0e-12,160e-12,9,endpoint=True));
ax.set_xticks(np.linspace(0e-12,160e-12,17,endpoint=True),minor=True);
ax.set_title('Inverter - Transient Simulation')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Signal [V]')
ax.plot(analysis.time,analysis.I)
ax.plot(analysis.time,analysis.O)
ax.legend(('Input (I)', 'Output (O)'))
plt.tight_layout()
plt.show()

## Graphical editor

_Magic_ can be started specifying only one file (`.magicrc`) that sets everything up (layers, macros, DRC rules, etc.)! Both a console and a GUI are started. Some functions are only accessible using the command-line interface or using keybindings.

In [ ]:
!magic -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc

# To have fun going further...

Here are a couple ideas to spend a good time exploring those beautiful pieces of software:

 * Characterize an NMOS device or a PMOS device by applying voltage pulses on the different pads.
 * Start _Magic_ on another PDK and design another inverter.
 * Vary the devices' geometry; try to make the pull-up and the pull-down waveforms symmetrical.
 
 **Do not hesitate to open the `.mag` files to see how they are built and to adapt from them!**
 
 > Good luck and read the docs. 😉

# References

Inspired from:
“Silicon Notebooks.” CHIPS Alliance, Apr. 08, 2023. Accessed: Apr. 10, 2023. [Online]. Available: https://github.com/chipsalliance/silicon-notebooks/blob/b65134a43b01ae31423f7ee87110740b2257ac42/analog-inverter-magic.ipynb (Apache License 2.0)